In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import json

In [2]:
directory_in = './file/1.part_number'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：116



,No,Part Number,Url
0,1,CCR-3,https://www.standardbrand.com/en/ecatalog?part...
1,2,CCR-6,https://www.standardbrand.com/en/ecatalog?part...
2,3,CCR-7,https://www.standardbrand.com/en/ecatalog?part...
3,4,DS-2282,https://www.standardbrand.com/en/ecatalog?part...
4,5,LS-312,https://www.standardbrand.com/en/ecatalog?part...
...,...,...,...
111,112,SLS-219,https://www.standardbrand.com/en/ecatalog?part...
112,113,SLS-321,https://www.standardbrand.com/en/ecatalog?part...
113,114,SLS-349,https://www.standardbrand.com/en/ecatalog?part...
114,115,SLS-385,https://www.standardbrand.com/en/ecatalog?part...


In [4]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                             1
 Part Number                                                CCR-3
 Url            https://www.standardbrand.com/en/ecatalog?part...
 Name: 0, dtype: object)

In [5]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request, url_check = f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part={crawler_series['Part Number']}&func=PART&vid=''', crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            dict_dict = resp.json()

            if dict_dict:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_dict

1


{'pd': {'rowId': 1,
  'site': 'STD',
  'webBase': 'CCR3',
  'brand': 'STI',
  'basePart': 'CCR-3',
  'partDesc_en': 'Clutch Starter Safety / Cruise Release Switch',
  'partLongDesc_en': 'Clutch Starter Safety / Cruise Release Switch',
  'pop': 'F',
  'partComment': None,
  'qtyEach': 1,
  'webColumn_Id': None,
  'categoryName_en': None,
  'vehicleId': None,
  'mfg': None,
  'mfG_Part': None,
  'maxRows': 1,
  'brandLink': None,
  'imageUrl': None},
 'pp': {'buyersGuides': [{'basePart': 'CCR-3',
    'buyersGuideId': 29304,
    'buyersGuideDesc': 'Kia Sephia (01-94)'},
   {'basePart': 'CCR-3',
    'buyersGuideId': 31841,
    'buyersGuideDesc': 'Kia Spectra (04-00)'},
   {'basePart': 'CCR-3',
    'buyersGuideId': 30432,
    'buyersGuideDesc': 'Kia Sportage (00-95)'}],
  'partSpecs': [{'attributeCode': '110',
    'attributeValue': 1905,
    'attributeSort': 0,
    'attributeName_en': 'Warranty',
    'siteValue_en': '3 years/36,000 mile'},
   {'attributeCode': '9',
    'attributeValue': 475

In [6]:
dict_info = {key.strip().title(): value.strip() if type(value) == str else value for key, value in dict_dict['pd'].items() if key not in ['rowId', 'site', 'webBase', 'basePart', 'maxRows'] and value != None}

dict_info

{'Brand': 'STI',
 'Partdesc_En': 'Clutch Starter Safety / Cruise Release Switch',
 'Partlongdesc_En': 'Clutch Starter Safety / Cruise Release Switch',
 'Pop': 'F',
 'Qtyeach': 1}

In [7]:
vehicle = '\n'.join([dict_['buyersGuideDesc'].strip() for dict_ in dict_dict['pp']['buyersGuides']])

print(vehicle)

Kia Sephia (01-94)
Kia Spectra (04-00)
Kia Sportage (00-95)


In [8]:
list_dict, dict_count = dict_dict['pp']['partSpecs'], {}
for dict_ in list_dict:
    key = dict_['attributeName_en'].strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_specification, dict_no = {}, {}
for i, dict_ in enumerate(list_dict):
    key = dict_['attributeName_en'].strip()
    if dict_count[key] == 1:
        dict_specification[str(i)] = {key: dict_['siteValue_en'].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_specification[str(i)] = {f'{key}-{dict_no[key]}': dict_['siteValue_en'].strip()}

dict_specification

{'0': {'Warranty': '3 years/36,000 mile'},
 '1': {'Connector Gender': 'Female'},
 '2': {'Terminal Quantity': '2'},
 '3': {'Terminal Gender': 'Male'},
 '4': {'Terminal Type': 'Blade'}}

In [9]:
dict_type, list_dict = {}, dict_dict['pp']['categories']
for i, dict_ in enumerate(list_dict):
    dict_type[str(i)] = dict_['categoryName_en'].strip()

dict_type

{'0': 'Switch - Clutch Starter Safety', '1': 'Switch - Cruise Control Release'}

In [10]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Part Number': crawler_series['Part Number'],
                          'Vehicle': vehicle,
                          'Picture': '',
                          'Url': crawler_series['Url'],
                          'Json_Type': json.dumps(dict_type),
                          'Json_Info': json.dumps(dict_info),
                          'Json_Specification': json.dumps(dict_specification)})

list_dict_correct

[{'No': 1,
  'Part Number': 'CCR-3',
  'Vehicle': 'Kia Sephia (01-94)\nKia Spectra (04-00)\nKia Sportage (00-95)',
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=CCR3&type=p&search=s',
  'Json_Type': '{"0": "Switch - Clutch Starter Safety", "1": "Switch - Cruise Control Release"}',
  'Json_Info': '{"Brand": "STI", "Partdesc_En": "Clutch Starter Safety / Cruise Release Switch", "Partlongdesc_En": "Clutch Starter Safety / Cruise Release Switch", "Pop": "F", "Qtyeach": 1}',
  'Json_Specification': '{"0": {"Warranty": "3 years/36,000 mile"}, "1": {"Connector Gender": "Female"}, "2": {"Terminal Quantity": "2"}, "3": {"Terminal Gender": "Male"}, "4": {"Terminal Type": "Blade"}}'}]

In [11]:
crawler_status = True

crawler_status

True

In [12]:
crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Part Number                                                CCR-3
 Url            https://www.standardbrand.com/en/ecatalog?part...
 Request_Url    https://ecatalog.smpcorp.com/V2/STD/api/part/p...
 Check_Url      https://www.standardbrand.com/en/ecatalog?part...
 Name: 0, dtype: object]

In [13]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct

,No,Part Number,Vehicle,Url,Json_Type,Json_Info,Json_Specification
0,1,CCR-3,Kia Sephia (01-94)\nKia Spectra (04-00)\nKia S...,https://www.standardbrand.com/en/ecatalog?part...,"{""0"": ""Switch - Clutch Starter Safety"", ""1"": ""...","{""Brand"": ""STI"", ""Partdesc_En"": ""Clutch Starte...","{""0"": {""Warranty"": ""3 years/36,000 mile""}, ""1""..."


In [14]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)

df_error

,No,Part Number,Url,Request_Url,Check_Url
0,1,CCR-3,https://www.standardbrand.com/en/ecatalog?part...,https://ecatalog.smpcorp.com/V2/STD/api/part/p...,https://www.standardbrand.com/en/ecatalog?part...
